In [1]:
from ROOT import TFile
from ROOT import TCanvas, TPad, TLegend, TLatex
from rtColors import colorWheel

Welcome to JupyROOT 6.26/00


In [83]:
class Canvas():
    def __init__(self, configs=None):
        self.cvs = TCanvas("c", "", 720, 900)
        self.pad_up = TPad("up", "", 0, 0.25, 1, 1)
        self.pad_up.SetGrid(True)
        self.pad_up.SetBottomMargin(0.02)
        self.pad_down = TPad("down", "", 0, 0, 1, 0.25)
        self.pad_down.SetGrid(True)
        self.pad_down.SetTopMargin(0.08)
        self.pad_down.SetBottomMargin(0.3)
        
        self.legend = TLegend(0.45, 0.05, 0.9, 0.43)
        self.legend.SetFillStyle(0)
        self.legend.SetBorderSize(0)
        self.legend.SetNColumns(2)
        
        self.info = TLatex()
        self.info.SetTextSize(0.03)
        self.info.SetTextFont(42)
        self.logo = TLatex()
        self.logo.SetTextSize(0.04)
        self.logo.SetTextFont(61)
        self.extra_logo = TLatex()
        self.extra_logo.SetTextSize(0.035)
        self.extra_logo.SetTextFont(52)
        
        self.configs = configs
        
    def get_hists(self, eff_pog, eff_rep):
        self.eff_pog = eff_pog
        self.eff_rep = eff_rep
        
        # make difference
        eff_diff = eff_rep.Clone("diff")
        for bin in range(eff_diff.GetNbinsX()):
            this_eff_pog = eff_pog.GetBinContent(bin)
            this_eff_rep = eff_rep.GetBinContent(bin)
            this_err = eff_pog.GetBinError(bin)
            if this_err == 0:
                eff_diff.SetBinContent(bin, 0)
            else:
                eff_diff.SetBinContent(bin, (this_eff_rep-this_eff_pog)/this_err)
        self.eff_diff = eff_diff
        self.__decorate()
    
    def __decorate(self):
        black = colorWheel['kBlack']
        blue = colorWheel['kBlue']
        red = colorWheel['kRed']
        
        self.eff_rep.SetStats(0)
        self.eff_rep.SetTitle("")
        self.eff_rep.GetXaxis().SetTitleSize(0)
        self.eff_rep.GetXaxis().SetLabelSize(0)
        self.eff_rep.GetXaxis().SetRangeUser(10., 500.)
        self.eff_rep.GetYaxis().SetTitle("efficiency")
        self.eff_rep.GetYaxis().SetTitleOffset(1.25)
        self.eff_rep.GetYaxis().SetRangeUser(0.4, 1.)
        self.eff_rep.SetLineColor(red)
        self.eff_rep.SetMarkerColor(red)
        self.eff_rep.SetMarkerStyle(21)
        self.eff_rep.SetMarkerSize(1)
        
        self.eff_pog.SetStats(0)
        self.eff_pog.SetTitle("")
        self.eff_pog.GetXaxis().SetTitleSize(0)
        self.eff_rep.GetXaxis().SetLabelSize(0)
        self.eff_pog.GetXaxis().SetRangeUser(10., 500.)
        self.eff_pog.GetYaxis().SetTitle("efficiency")
        self.eff_pog.GetYaxis().SetTitleOffset(1.25)
        self.eff_pog.GetYaxis().SetRangeUser(0.4, 1.)
        self.eff_pog.SetLineColor(blue)
        self.eff_pog.SetMarkerColor(blue)
        self.eff_pog.SetMarkerStyle(21)
        self.eff_pog.SetMarkerSize(1)
        
        self.eff_diff.SetStats(0)
        self.eff_diff.SetTitle("")
        self.eff_diff.GetXaxis().SetTitle("p_{T} [GeV]")
        self.eff_diff.GetXaxis().SetTitleSize(0.1)
        self.eff_diff.GetXaxis().SetTitleOffset(0.9)
        self.eff_diff.GetXaxis().SetLabelSize(0.08)
        self.eff_diff.GetXaxis().SetRangeUser(10., 200.)
        self.eff_diff.GetYaxis().SetTitle("(Eff_{REP}-Eff_{POG})/#sigma_{POG}")
        self.eff_diff.GetYaxis().CenterTitle()
        self.eff_diff.GetYaxis().SetTitleSize(0.1)
        self.eff_diff.GetYaxis().SetTitleOffset(0.4)
        self.eff_diff.GetYaxis().SetLabelSize(0.08)
        self.eff_diff.GetYaxis().SetRangeUser(-2.2, 2.2)
        self.eff_diff.SetLineColor(black)
        self.eff_diff.SetMarkerColor(black)
        self.eff_diff.SetMarkerStyle(20)
        self.eff_diff.SetMarkerSize(1)
        
    def combine(self):
        self.pad_up.cd()
        self.pad_up.SetLogx()
        self.eff_pog.Draw("p&same")
        self.eff_rep.Draw("p&same")
        self.legend.Draw("same")
        if "info" in self.configs.keys():
            self.info.DrawLatexNDC(0.74, 0.91, self.configs['info'])
        self.logo.DrawLatexNDC(0.15, 0.83, "CMS")
        self.extra_logo.DrawLatexNDC(0.15, 0.78, "Work in progress")
        
        if "is_data" in self.configs.keys():
            if not self.configs['is_data']:
                sim_latex = TLatex()
                sim_latex.SetTextSize(0.035)
                sim_latex.SetTextFont(42)
                sim_latex.DrawLatexNDC(0.62, 0.85, "Simulation")
        
        if "eta_range" in self.configs.keys():
            eta_low = self.configs['eta_range'][0]
            eta_high = self.configs['eta_range'][1]
            text = f"{eta_low} < #eta < {eta_high}"
            eta_latex = TLatex()
            eta_latex.SetTextSize(0.035)
            eta_latex.SetTextFont(52)
            eta_latex.DrawLatexNDC(0.62, 0.80, text)
                
        self.pad_down.cd()
        self.pad_down.SetLogx()
        self.eff_diff.Draw("p")
        
        self.cvs.cd()
        self.pad_up.Draw()
        self.pad_down.Draw()
        
    def draw(self):
        self.cvs.Draw()
    
    def savefig(self, figname):
        self.cvs.SaveAs(figname)

In [95]:
ERA = "2018"
is_data = False
eta_bins = [-2.5, -2., -1.566, -1.444, -0.8, 0., 0.8, 1.444, 1.566, 2., 2.5]
pt_bins = [10, 20, 35, 50, 100, 200, 500]

def get_eff_hists(era, is_data):
    f_pog = TFile.Open(f"{era}/Electron/efficiency_POG.root")
    f_rep = TFile.Open(f"HcToWAElectron/{era}_MVANoIsoWP90/efficiency.root")
    
    if is_data:
        h_pog = f_pog.Get("EGamma_EffData2D"); h_pog.SetDirectory(0)
        h_rep = f_rep.Get("data"); h_rep.SetDirectory(0)
    else:
        h_pog = f_pog.Get("EGamma_EffMC2D"); h_pog.SetDirectory(0)
        h_rep = f_rep.Get("sim"); h_rep.SetDirectory(0)
    
    return (h_pog, h_rep)

h_pog_2d, h_rep_2d = get_eff_hists(ERA, is_data)
luminosity = {
    "2016preVFP": "19.8 /fb",
    "2016postVFP": "19.8 /fb",
    "2017": "41.5 /fb",
    "2018": "59.7 /fb"
}
for bin in range(1, len(eta_bins)-1):
    h_pog = h_pog_2d.ProjectionY(f"x_pog_{bin}", bin, bin)
    h_rep = h_rep_2d.ProjectionY(f"x_rep_{bin}", bin, bin)
    configs = {
        "era": ERA,
        "is_data": is_data,
        "info":  "L^{int} = " + f"{luminosity[ERA]}",
        "eta_range": [eta_bins[bin], eta_bins[bin+1]]
    }
    cvs = Canvas(configs=configs)
    cvs.get_hists(h_pog, h_rep)
    cvs.combine()
    cvs.savefig(f"../www/HcToWA/UltraLegacy/MetaInfo/{ERA}/Electron/ReproducePOG/eff_{eta_bins[bin]}to{eta_bins[bin+1]}_{is_data}.png")

Warning in <TCanvas::Constructor>: Deleting canvas with same name: c
Info in <TCanvas::Print>: png file ../www/HcToWA/UltraLegacy/MetaInfo/2018/Electron/ReproducePOG/eff_-2.0to-1.566_False.png has been created
Warning in <TCanvas::Constructor>: Deleting canvas with same name: c
Info in <TCanvas::Print>: png file ../www/HcToWA/UltraLegacy/MetaInfo/2018/Electron/ReproducePOG/eff_-1.566to-1.444_False.png has been created
Warning in <TCanvas::Constructor>: Deleting canvas with same name: c
Info in <TCanvas::Print>: png file ../www/HcToWA/UltraLegacy/MetaInfo/2018/Electron/ReproducePOG/eff_-1.444to-0.8_False.png has been created
Warning in <TCanvas::Constructor>: Deleting canvas with same name: c
Info in <TCanvas::Print>: png file ../www/HcToWA/UltraLegacy/MetaInfo/2018/Electron/ReproducePOG/eff_-0.8to0.0_False.png has been created
Warning in <TCanvas::Constructor>: Deleting canvas with same name: c
Info in <TCanvas::Print>: png file ../www/HcToWA/UltraLegacy/MetaInfo/2018/Electron/Reproduc